# [백준/웜홀](https://www.acmicpc.net/problem/1865)

## 풀이과정

### 첫번째 시도

#### 풀이과정
벨만 포드를 적용시켰으나 사이클이 첫번째 노드를 포함하지 않는다면 사이클을 검출할 수 없기에 오답이 나왔습니다.

In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    inf = 250000001
    for _ in range(int(input())):
        n, m, w = map(int, input().split())
        edges = [
            tuple(map(int, input.split())) for _, input in zip(range(m), sys.stdin)
        ]
        edges += [(e, s, t) for s, e, t in edges]
        edges += [
            (s, e, -t)
            for s, e, t in (
                map(int, input.split()) for _, input in zip(range(w), sys.stdin)
            )
        ]
        from1 = [0, 0] + [inf] * (n - 1)
        for _ in range(n + 1):
            is_updated = False
            for s, e, t in edges:
                if from1[s] == inf:
                    continue
                new = from1[s] + t
                if from1[e] > new:
                    from1[e] = new
                    is_updated = True
            if not is_updated:
                break
        print("YES" if is_updated else "NO")


solution()

### 두번째 시도

#### 풀이과정
첫번째 풀이에서 첫번째 노드 뿐만이 아니라 모든 노드를 검사하도록 만들었지만 이로 인하여 시간이 매우 증가하여 시간 초과가 났습니다.

In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    inf = 250000001
    for _ in range(int(input())):
        n, m, w = map(int, input().split())
        edges = {}
        for _, input_ in zip(range(m), sys.stdin):
            s, e, t = map(int, input_.split())
            s, e = s - 1, e - 1
            if (s, e) in edges:
                edges[s, e] = edges[e, s] = min(t, edges[s, e])
            else:
                edges[s, e] = edges[e, s] = t
        for _, input_ in zip(range(w), sys.stdin):
            s, e, t = map(int, input_.split())
            s, e, t = s - 1, e - 1, -t
            if (s, e) in edges:
                edges[s, e] = min(t, edges[s, e])
            else:
                edges[s, e] = t
        graph = [
            [edges[s, e] if (s, e) in edges else inf if s != e else 0 for e in range(n)]
            for s in range(n)
        ]
        for _ in range(n):
            is_updated = False
            for (s, e), t in edges.items():
                for nodeto in graph:
                    if nodeto[s] == inf:
                        continue
                    new = nodeto[s] + t
                    if nodeto[e] > new:
                        nodeto[e] = new
                        is_updated = True
            if not is_updated:
                break
        print("YES" if is_updated else "NO")


solution()

### 세번째 시도

#### 풀이과정
두번쨰 풀이에서 시작점을 모든 점으로 잡지 않고 들렀던 점을 저장하는 집합을 만든 뒤, 만약 어떤 점이 현재 시작점으로 포함되는 점과 쌍방으로 연결되어 있다면 해당 점 또한 포함시켰습니다. 이후, 들렀던 점이 아닌 경우에만 검사해서 효율성을 증가시켰습니다.

In [ ]:
def solution():
    import sys

    input = sys.stdin.readline
    inf = 250000001
    for _ in range(int(input())):
        n, m, w = map(int, input().split())
        edges = {}
        for _, input_ in zip(range(m), sys.stdin):
            # 쌍방향 도로
            s, e, t = map(int, input_.split())
            s, e = s - 1, e - 1
            if (s, e) in edges:
                # 이미 있는 경우 더 적게 걸리는 도로만 저장
                edges[s, e] = edges[e, s] = min(t, edges[s, e])
            else:
                # 없으면 그대로 저장합니다.
                edges[s, e] = edges[e, s] = t
        for _, input_ in zip(range(w), sys.stdin):
            # 단방향 웜홀
            s, e, t = map(int, input_.split())
            s, e, t = s - 1, e - 1, -t
            if (s, e) in edges:
                # 이미 도로나 웜홀이 있는 경우 더 적게 걸리는 웜홀을 저장합니다.
                edges[s, e] = min(t, edges[s, e])
            else:
                # 없으면 바로 저장합니다.
                edges[s, e] = t
        # 점에서 점으로 가는 최소 비용을 저장하는 리스트
        graph = [
            [edges[s, e] if (s, e) in edges else inf if s != e else 0 for e in range(n)]
            for s in range(n)
        ]
        # 들렀던 점을 저장하는 집합
        visited = set()
        for node, nodeto in enumerate(graph):
            if node in visited:
                # 이미 들렀던 점일 경우 생략합니다.
                continue
            # 현재 점을 저장합니다.
            visited.add(node)
            for _ in range(n):
                # 변한 적이 있는지 체크하는 부울 값
                is_updated = False
                for (s, e), t in edges.items():
                    if nodeto[s] == inf:
                        # 만약 현재 점에서 경로의 시작점까지의 거리가
                        # 무한대일 경우 생략합니다.
                        continue
                    # 현재 점에서 경로의 시작점까지의 거리와
                    # 경로의 거리를 더하여 새 거리를 만듭니다.
                    new = nodeto[s] + t
                    if nodeto[e] > new:
                        # 만약 새 거리가 기존 거리보다 작다면
                        # 새 거리를 저장합니다.
                        nodeto[e] = new
                        # 변한 적이 있으므로 값을 참으로 바꿉니다.
                        is_updated = True
                if not is_updated:
                    # 변한 적이 없다면 더이상 변할 필요가 없으므로 반복을 종료합니다.
                    break
            if is_updated:
                # 벨만-포드 알고리즘의 최대 경로인
                # n번의 경로를 돌았음에도 변한 적이 있었다면
                # 무한히 경로가 작아지는 경우가 있다는 뜻이므로
                # 반복을 멈춥니다.
                break
            for i in range(n):
                if graph[i][node] < inf:
                    # 어떤 점에서 현재 점으로 들어오는 거리가 무한보다 작다면
                    # 현재 점과 연결되어 있다는 뜻이므로
                    # 들렀던 점에 포함시킵니다.
                    visited.add(i)
        # 한 번이라도 n 번 이상 검사했을 때까지 변한 점이 있다면
        # 무한히 거리가 줄어드는 경로가 존재한다는 뜻이므로
        # 변한 적이 있다면 "YES" 아니라면 "NO"를 출력
        print("YES" if is_updated else "NO")


solution()

## 해답

In [108]:
def solution():
    import sys

    input = sys.stdin.readline
    inf = 250000001
    for _ in range(int(input())):
        n, m, w = map(int, input().split())
        edges = {}
        for _, input_ in zip(range(m), sys.stdin):
            s, e, t = map(int, input_.split())
            s, e = s - 1, e - 1
            if (s, e) in edges:
                edges[s, e] = edges[e, s] = min(t, edges[s, e])
            else:
                edges[s, e] = edges[e, s] = t
        for _, input_ in zip(range(w), sys.stdin):
            s, e, t = map(int, input_.split())
            s, e, t = s - 1, e - 1, -t
            if (s, e) in edges:
                edges[s, e] = min(t, edges[s, e])
            else:
                edges[s, e] = t
        graph = [
            [edges[s, e] if (s, e) in edges else inf if s != e else 0 for e in range(n)]
            for s in range(n)
        ]
        visited = set()
        for node, nodeto in enumerate(graph):
            if node in visited:
                continue
            visited.add(node)
            for _ in range(n):
                is_updated = False
                for (s, e), t in edges.items():
                    if nodeto[s] == inf:
                        continue
                    new = nodeto[s] + t
                    if nodeto[e] > new:
                        nodeto[e] = new
                        is_updated = True
                if not is_updated:
                    break
            if is_updated:
                break
            for i in range(n):
                if graph[i][node] < inf:
                    visited.add(i)
        print("YES" if is_updated else "NO")

## 예제

In [109]:
# black 모듈을 jupyter notebook 내에서 사용 가능하게 만들어주는 blackcellmagic 모듈 불러오기
%load_ext blackcellmagic
# %%black

# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [110]:
exmaples = [
    """4 4 1
1 4 1
4 1 1
3 1 1
3 2 1
2 3 2""",
    """4 3 1
1 2 1
2 1 1
3 4 1
4 3 2""",
]
exmaple = "\n".join([str(len(exmaples))] + exmaples)
test_solution(exmaple)
# answer:
# YES
# YES

YES
YES


In [111]:
test_solution(
    """2
3 3 1
1 2 2
1 3 4
2 3 1
3 1 3
3 2 1
1 2 3
2 3 4
3 1 8
"""
)
# answer:
# NO
# YES

NO
YES
